In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ieee-fraud-detection/sample_submission.csv
/kaggle/input/ieee-fraud-detection/test_identity.csv
/kaggle/input/ieee-fraud-detection/train_identity.csv
/kaggle/input/ieee-fraud-detection/test_transaction.csv
/kaggle/input/ieee-fraud-detection/train_transaction.csv


In [2]:
!pip install mlflow dagshub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.0/29.0 MB 50.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 84.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.1/260.1 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 692.3/692.3 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.3/74.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.4 MB/s eta 0:00:00
  Attempting uninstall: dacite
    Found existing installation: dacite 1.9.2
    Uninstalling dacite-1.9.2:
      

In [3]:
import dagshub
dagshub.init(repo_owner='lkata22',
             repo_name='IEEE-CIS-Fraud-Detection',
             mlflow=True)

import mlflow

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=083d9773-08b9-4a36-bbd2-4936e2a81961&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=4cbf4e780ce2d8db709daee82fc95a6e0465da98a2a817677bc26f5e0b9b709f




Output()

Accessing as lkata22

Initialized MLflow to track repo "lkata22/IEEE-CIS-Fraud-Detection"

Repository lkata22/IEEE-CIS-Fraud-Detection initialized!

### Imports


In [4]:
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import gc
import warnings
warnings.filterwarnings('ignore')
from sklearn.impute import SimpleImputer
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression








## Memory Reducer to reduce memory usage of DataFrame

In [5]:
class MemoryReducer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X = reduce_mem_usage(X)
        return X

def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtype
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose:
        print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(
            end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

## Cleaner Class

In [6]:
class MissingValueHandler(BaseEstimator, TransformerMixin):
    def __init__(self, num_strategy='median', cat_strategy='constant', fill_value='missing'):
        self.num_strategy = num_strategy
        self.cat_strategy = cat_strategy
        self.fill_value = fill_value
        self.num_impute_values = {}
        self.cat_impute_values = {}
        
    def fit(self, X, y=None):
        num_cols = [col for col in X.columns if X[col].dtype != 'object' 
                   and col not in ['isFraud', 'TransactionID']]
        
        if self.num_strategy == 'median':
            for col in num_cols:
                self.num_impute_values[col] = X[col].median()
        elif self.num_strategy == 'mean':
            for col in num_cols:
                self.num_impute_values[col] = X[col].mean()
        elif self.num_strategy == 'constant':
            for col in num_cols:
                self.num_impute_values[col] = 0 
                
        cat_cols = [col for col in X.columns if X[col].dtype == 'object']
        if self.cat_strategy == 'missing':
            for col in cat_cols:
                self.cat_impute_values[col] = 'missing'
        elif self.cat_strategy == 'mode':
            for col in cat_cols:
                self.cat_impute_values[col] = X[col].mode()[0]
        elif self.cat_strategy == 'constant':
            for col in cat_cols:
                self.cat_impute_values[col] = self.fill_value
                
        return self
    
    def transform(self, X, y=None):
        for col, val in self.num_impute_values.items():
            if col in X.columns:
                X[col].fillna(val, inplace=True)
                X[f'{col}_missing'] = X[col].isna().astype(int)
                
        for col, val in self.cat_impute_values.items():
            if col in X.columns:
                X[col].fillna(val, inplace=True)
                
        return X

## Feature Engineer Class (Creates new features and transforms existing ones


In [7]:
class FeatureEngineer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.freq_encoders = {}
        
    def fit(self, X, y=None):
        freq_cols = ['card1', 'card2', 'card3', 'card5', 'addr1', 'addr2']
        for col in freq_cols:
            if col in X.columns:
                self.freq_encoders[col] = X[col].value_counts(dropna=False)
        return self
    
    def transform(self, X, y=None):
        if 'TransactionAmt' in X.columns:
            for group_col in ['card1', 'card4']:
                if group_col in X.columns:
                    group_means = X.groupby([group_col])['TransactionAmt'].mean()
                    X[f'TransactionAmt_to_mean_{group_col}'] = X['TransactionAmt'] / X[group_col].map(group_means)
        
        for col, counts in self.freq_encoders.items():
            if col in X.columns:
                X[f'{col}_freq'] = X[col].map(counts)
        
        if 'P_emaildomain' in X.columns and 'R_emaildomain' in X.columns:
            X['P_emaildomain_match_R_emaildomain'] = (X['P_emaildomain'] == X['R_emaildomain']).astype(int)
            
        return X

## Handles unseen categories and missing values safely

In [8]:
class CategoricalEncoder(BaseEstimator, TransformerMixin):
  
    def __init__(self, handle_unseen='impute', impute_value='missing'):
        self.handle_unseen = handle_unseen 
        self.impute_value = impute_value
        self.encoders = {}
        self.known_categories = {}
        
    def fit(self, X, y=None):
        cat_cols = [col for col in X.columns if X[col].dtype == 'object']
        for col in cat_cols:
            unique_vals = X[col].dropna().unique()
            self.known_categories[col] = set(unique_vals)
            
            le = LabelEncoder()
            if self.handle_unseen == 'impute':
                le.fit(np.append(unique_vals, self.impute_value))
            else:
                le.fit(unique_vals)
            self.encoders[col] = le
        return self
    
    def transform(self, X, y=None):
        X = X.copy()
        for col, le in self.encoders.items():
            if col in X.columns:
                X[col] = X[col].astype(str)
                X[col] = X[col].replace('nan', self.impute_value)
                
                if self.handle_unseen == 'impute':
                    unseen_mask = ~X[col].isin(self.known_categories[col])
                    X.loc[unseen_mask, col] = self.impute_value
                
                try:
                    X[col] = le.transform(X[col])
                except ValueError:
                    unseen = set(X[col].unique()) - set(le.classes_)
                    X.loc[X[col].isin(unseen), col] = self.impute_value
                    X[col] = le.transform(X[col])
        return X

## Featire Selector Class (Selects features based on importance or other criteria)

In [9]:
class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, strategy='default'):
        self.strategy = strategy
        self.selected_features = None
        
    def fit(self, X, y=None):
        if self.strategy == 'default':
            cols_to_drop = [
                'TransactionID', 
                'V300', 'V309', 'V111', 'V124', 'V106', 'V125', 'V315', 'V134', 'V102', 'V123', 
                'V136', 'V305', 'V110', 'V129', 'V114', 'V116', 'V298', 'V126', 'V113', 'V105', 
                'V119', 'V104', 'V122', 'V320', 'V115', 'V317', 'V303', 'V112', 'V118', 'V108', 
                'V127', 'V132', 'V109', 'V103', 'V120', 'V107', 'V131', 'V135', 'V308', 'V117', 
                'V121', 'V133', 'V130', 'V318', 'V304', 'V128', 'V319', 'V307', 'V306', 'V302', 
                'V311', 'V301', 'V310'
            ]
            self.selected_features = [col for col in X.columns if col not in cols_to_drop]
            
        return self
    
    def transform(self, X, y=None):
        if self.selected_features is not None:
            X = X.copy()
            for col in self.selected_features:
                if col not in X.columns:
                    X[col] = np.nan 
            return X[self.selected_features]
        return X

## Start of training

In [10]:
RANDOM_STATE = 42
N_FOLDS = 5
TEST_SIZE = 0.2


train_identity = pd.read_csv("/kaggle/input/ieee-fraud-detection/train_identity.csv")
train_transaction = pd.read_csv("/kaggle/input/ieee-fraud-detection/train_transaction.csv")


train_df = pd.merge(train_transaction, train_identity, on='TransactionID', how='left')

print(f"Training data shape: {train_df.shape}")
print(f"Fraud rate: {train_df['isFraud'].mean():.4f}")

X_train, X_val, y_train, y_val = train_test_split(
    train_df.drop('isFraud', axis=1),
    train_df['isFraud'],
    test_size=TEST_SIZE,
    random_state=RANDOM_STATE,
    stratify=train_df['isFraud']
)


print("Building preprocessing pipeline...")
preprocessing_pipeline = Pipeline([
    ('memory_reducer', MemoryReducer()),
    ('missing_handler', MissingValueHandler(
        num_strategy='median',
        cat_strategy='constant',
        fill_value='missing'
    )),
    ('feature_engineer', FeatureEngineer()),
    ('categorical_encoder', CategoricalEncoder(
        handle_unseen='impute',
        impute_value='missing'
    )),
    ('feature_selector', FeatureSelector(strategy='default'))
])

print("Preprocessing training data...")
X_train_preprocessed = preprocessing_pipeline.fit_transform(X_train, y_train)
X_val_preprocessed = preprocessing_pipeline.transform(X_val)

X_train_preprocessed = X_train_preprocessed.fillna(-999)
X_val_preprocessed = X_val_preprocessed.fillna(-999)

print("Training Logistic Regression model...")

kf = KFold(n_splits=N_FOLDS, shuffle=True, random_state=RANDOM_STATE)

cv_scores = []
models = []

for fold, (train_idx, valid_idx) in enumerate(kf.split(X_train_preprocessed, y_train)):
    print(f"\nFold {fold + 1}/{N_FOLDS}")
    
    X_tr, X_v = X_train_preprocessed.iloc[train_idx], X_train_preprocessed.iloc[valid_idx]
    y_tr, y_v = y_train.iloc[train_idx], y_train.iloc[valid_idx]
    
    model = LogisticRegression(
        penalty='l2',
        solver='lbfgs',  
        max_iter=500,         
        random_state=RANDOM_STATE,
        n_jobs=-1         
    )
    
    model.fit(X_tr, y_tr)  

    val_preds = model.predict_proba(X_v)[:, 1]
    score = roc_auc_score(y_v, val_preds)
    cv_scores.append(score)
    models.append(model)
    print(f"Fold {fold + 1} AUC: {score:.5f}")

print("\nCross-validation results:")
print(f"Mean AUC: {np.mean(cv_scores):.5f}")
print(f"Std AUC: {np.std(cv_scores):.5f}")

print("\nEvaluating on holdout validation set...")
val_preds = np.mean([model.predict_proba(X_val_preprocessed)[:, 1] for model in models], axis=0)
val_score = roc_auc_score(y_val, val_preds)
print(f"Validation AUC: {val_score:.5f}")





Training data shape: (590540, 434)
Fraud rate: 0.0350
Building preprocessing pipeline...
Preprocessing training data...
Mem. usage decreased to 838.92 Mb (46.4% reduction)
Mem. usage decreased to 209.73 Mb (46.4% reduction)
Training Logistic Regression model...

Fold 1/5


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Fold 1 AUC: 0.65313

Fold 2/5


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Fold 2 AUC: 0.66127

Fold 3/5
Fold 3 AUC: 0.65838

Fold 4/5
Fold 4 AUC: 0.65008

Fold 5/5
Fold 5 AUC: 0.64437

Cross-validation results:
Mean AUC: 0.65345
Std AUC: 0.00599

Evaluating on holdout validation set...
Validation AUC: 0.65578



## Feature Importance


In [12]:

print("\nFeature importance analysis...")
feature_importance = pd.DataFrame({
    'feature': X_train_preprocessed.columns,
    'importance': np.mean([np.abs(model.coef_[0]) for model in models], axis=0)
}).sort_values('importance', ascending=False)


print("\nTop 20 features:")
print(feature_importance.head(20))


Feature importance analysis...

Top 20 features:
        feature  importance
378  DeviceInfo    0.000036
3         card1    0.000030
782  card1_freq    0.000030
281        V264    0.000024
331        V332    0.000023
282        V265    0.000022
220        V203    0.000020
332        V333    0.000018
280        V263    0.000018
291        V274    0.000017
182        V165    0.000017
43          D15    0.000016
292        V275    0.000016
318        V314    0.000016
316        V312    0.000015
29           D1    0.000014
30           D2    0.000014
32           D4    0.000013
290        V273    0.000013
330        V331    0.000013


## MLFlow logging

In [14]:
mlflow.set_experiment("LR_Training")

with mlflow.start_run(run_name="LR_Cleaning"):
    mlflow.log_param("memory_reduction", True)
    mlflow.log_param("missing_value_strategy_num", "median")
    mlflow.log_param("missing_value_strategy_cat", "constant_missing")
    mlflow.log_text("Used MemoryReducer and MissingValueHandler for missing values.", "cleaning_notes.txt")

mlflow.end_run()

with mlflow.start_run(run_name="LR_Feature_Engineering"):
    mlflow.log_param("feature_engineering_applied", True)
    mlflow.log_param("freq_encoding_columns", ['card1', 'card2', 'card3', 'card5', 'addr1', 'addr2'])
    mlflow.log_text("Created TransactionAmt_to_mean features and Email domain matching.", "feature_engineering_notes.txt")

mlflow.end_run()

with mlflow.start_run(run_name="LR_Feature_Selection"):
    mlflow.log_param("feature_selection_strategy", "manual_drop_low_importance_V-features")
    mlflow.log_param("final_selected_features", len(preprocessing_pipeline.named_steps['feature_selector'].selected_features))
    mlflow.log_text("Dropped around 50 V-features manually based on prior EDA.", "feature_selection_notes.txt")

mlflow.end_run()

with mlflow.start_run(run_name="LR_Final_Model") as run:
    mlflow.log_params({
        "model_type": "LogisticRegression",
        "n_folds": N_FOLDS,
        "random_state": RANDOM_STATE,
        "penalty": "l2",
        "solver": "lbfgs",
        "max_iter": 500
    })
    mlflow.log_metrics({
        "fold_1_auc": cv_scores[0],
        "fold_2_auc": cv_scores[1],
        "fold_3_auc": cv_scores[2],
        "fold_4_auc": cv_scores[3],
        "fold_5_auc": cv_scores[4],
        "mean_auc": np.mean(cv_scores),
        "std_auc": np.std(cv_scores),
        "validation_auc": val_score
    })

    best_model_idx = np.argmax(cv_scores)
    best_model = models[best_model_idx]
    
    final_pipeline = Pipeline([
        ('preprocessing', preprocessing_pipeline),
        ('model', best_model)
    ])
    
    
    mlflow.sklearn.log_model(best_model, "best_RF_model")
    mlflow.sklearn.log_model(final_pipeline, "full_pipeline")

    top_features = feature_importance.head(20).to_dict()
    mlflow.log_dict(top_features, "feature_importance/top_20_features.json")
    
    val_preds_sample = pd.DataFrame({
        'actual': y_val,
        'predicted': val_preds
    }).sample(1000)
    mlflow.log_table(val_preds_sample, "validation_predictions_sample.json")

    mlflow.log_text(f"""
    - Train shape: {X_train_preprocessed.shape}
    - Fraud rate: {y_train.mean():.4f}
    - Best fold AUC: {cv_scores[best_model_idx]:.4f}
    - Validation AUC: {val_score:.4f}
    """, "training_summary.txt")

    print(f"Successfully logged to MLflow! Run ID: {run.info.run_id}")


🏃 View run LR_Cleaning at: https://dagshub.com/lkata22/IEEE-CIS-Fraud-Detection.mlflow/#/experiments/7/runs/803044f5af9d41caa405fc6fd0147440
🧪 View experiment at: https://dagshub.com/lkata22/IEEE-CIS-Fraud-Detection.mlflow/#/experiments/7
🏃 View run LR_Feature_Engineering at: https://dagshub.com/lkata22/IEEE-CIS-Fraud-Detection.mlflow/#/experiments/7/runs/2dd1dc11404c4fd99f5debd0dca6401f
🧪 View experiment at: https://dagshub.com/lkata22/IEEE-CIS-Fraud-Detection.mlflow/#/experiments/7
🏃 View run LR_Feature_Selection at: https://dagshub.com/lkata22/IEEE-CIS-Fraud-Detection.mlflow/#/experiments/7/runs/ed5fdfee1cb84be0a504008fab6a90f4
🧪 View experiment at: https://dagshub.com/lkata22/IEEE-CIS-Fraud-Detection.mlflow/#/experiments/7


2025/04/27 19:36:35 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/04/27 19:36:43 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Successfully logged to MLflow! Run ID: dc3d4e74ef774538af150379ecb2651b
🏃 View run LR_Final_Model at: https://dagshub.com/lkata22/IEEE-CIS-Fraud-Detection.mlflow/#/experiments/7/runs/dc3d4e74ef774538af150379ecb2651b
🧪 View experiment at: https://dagshub.com/lkata22/IEEE-CIS-Fraud-Detection.mlflow/#/experiments/7
